In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,UB40,Little By Little,Poor man's anger rising. \r\nThe ostrich hide...
1,Manowar,Pleasure Slave,She is waiting to kiss my hand \r\nBut she wi...
2,Demi Lovato,Shadow,I glanced upon the ground today I noticed some...
3,Adele,Last Nite,"Last night she said \r\nOh, baby \r\nI feel ..."
4,Christmas Songs,Miss You Most At Christmas,The fire is burning \r\nThe room's all aglow ...
5,David Allan Coe,"Whiskey, Whiskey (Take My Mind)","[Chorus] \r\nWhiskey, whiskey take my mind \..."
6,Olivia Newton-John,I Think I'll Say Goodbye,I never tried to change you \r\nBut the hurt ...
7,Irving Berlin,They Say It's Wonderful,They say that falling in love is wonderful \r...
8,Allman Brothers Band,Seven Turns,"Seven turns on the highway, \r\nSeven rivers ..."
9,Chris Brown,Blue Rose,"[Verse 1] \r\nGirl I double dare you, to keep..."


In [9]:
df['text'][0]

"Poor man's anger rising.  \r\nThe ostrich hides his head.  \r\nSoon the red blood will be boiling.  \r\nAnd blue blood will be dead.  \r\nWhile we say.  \r\n  \r\n[Chorus]  \r\nLittle by little by little,  \r\nAnd stone by stone.  \r\nRich man's mountain comes crumbling down  \r\n  \r\nPoor boy sleeps on straw,  \r\nThe rich boy sleeps in bed.  \r\nThat fat boy fills his belly,  \r\nMy poor boys's a dead,  \r\nWhile we say.  \r\n  \r\n[Chorus]  \r\n  \r\nLittle by little by little,  \r\nAnd stone by stone.  \r\nRich man's mountain comes crumbling down.  \r\n  \r\nThe rich man drives his car past,  \r\nThe poor man on bare feet.  \r\nThat rich man do get what he wants  \r\nThe poor must know defeat,  \r\nWhile we say.  \r\n  \r\n[Chorus]  \r\n  \r\nLittle by little by little,  \r\nAnd stone by stone.  \r\nRich man's mountain comes crumbling down.\r\n\r\n"

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [18]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\subra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.        , 0.00215382, 0.01692502, ..., 0.00538068, 0.02103184,
       0.05354274])

In [23]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [30]:
recommendation("Crying Over You")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))